In [7]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os

import pandas as pd

In [8]:
result = Path('result/')
report = Path('report/')
instances = Path('../data/uls')

In [9]:
MAX_CPU_TIME = 60.0
EPSILON = 1e-6

In [24]:
def readdata(datafile):
  
    with open(datafile, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # lendo o tamanho da instancia
    aux = 0
    N = 26
  
    # definindo vetores
    H = np.zeros(N)
    P = np.zeros(N)
    F = np.zeros(N)
    D = np.zeros(N)
  
    # lendo e armazenando dados
    aux = aux+1
    F[0] = float(linhas[aux])
    for i in range(1,N):
        F[i] = F[0]

    aux = aux+1
    H[0] = float(linhas[aux])
    for i in range(1,N):
        H[i] = H[0]

    aux = aux+1
    P[0] = float(linhas[aux])
    for i in range(1,N):
        P[i] = P[0]

    aux = aux+1
    D = np.fromstring(linhas[aux], dtype=float, sep = ' ')
 
    return N, H, P, F, D

In [25]:
def modelo(N, H, P, F, D, model):

    # adicionando variaveis
    x = model.addVars(N, name='x') 
    s = model.addVars(N, name='s')  
    y = model.addVars(N, vtype=GRB.BINARY, name='y') 

    # definindo funcao objetivo
    obj = 0
    for i in range(0, N):
        obj += P[i] * x[i]
        obj += H[i] * s[i]
        obj += F[i] * y[i]

    model.setObjective(obj, GRB.MINIMIZE)
  
    # definindo restrições
    model.addConstr(x[0] - s[0] == D[0])
    for i in range(1, N):
        model.addConstr(s[i-1] + x[i] - s[i] == D[i])
  
    for i in range(0, N):
        model.addConstr(x[i] - (D[i:N].sum())*y[i] <= 0)

    model.addConstr(s[N-1] == 0)

    # exportando modelo
    model.write("uls.lp")

    return model

In [26]:
def main(datafile):
    
    #datafile = "52_2.txt"

    # lendo os dados
    N, H, P, F, D = readdata(datafile)

    # criando o modelo
    model = gp.Model("uls")

    # configurando parametros
    model.Params.TimeLimit = MAX_CPU_TIME
    model.Params.MIPGap = EPSILON
    model.Params.method = 0 #-1=automatic, 0=primal, 1=dual , 2=barrier
    model.Params.NodeMethod = 1 #-1=automatic, 0=primal, 1=dual , 2=barrier
    model.Params.Threads = 1
    model.Params.OutputFlag = 0
    model.Params.Presolve = 0

    # model problem
    modelo(N, H, P, F, D, model)
    
    # optimize
    model.optimize()

    # get problem status
    tmp = 0
    if model.status == GRB.OPTIMAL:
        tmp = 1
    
    # imprimindo solucao
    #for var in model.getVars():
    #    print(f"{var.varName} = {var.x}")

    # get results
    objval = model.ObjVal
    objbound = model.ObjBound
    mipgap = model.MIPGap
    runtime = model.Runtime
    nodecount = model.NodeCount

    # imprimindo o valor otimo
    #print(f"objval: {round(objval,2)}")
    #print(f"objbound: {round(objbound,2)}")
    #print(f"runtime: {round(runtime,2)}")
    #print(f"mipgap: {round(mipgap,2)}")
    #print(f"nodecount: {round(nodecount,2)}")
    #print(f"status: {round(tmp,2)}")
    
    return objval, objbound, mipgap, runtime, nodecount, tmp

In [27]:
if __name__ == "__main__":

    for dim in [52]:
        for id in range(1,11):
            datafile = f"{instances}/{dim}_{id}.txt"
            print(f"Resolvendo instancia {datafile}")			
            
            objval, objbound, mipgap, runtime, numnode, tmp = main(datafile)

            arquivo = open(os.path.join(result,'uls_std_mip.csv'),'a')
            arquivo.write(datafile+';'
			   +str(round(objval,2))+';'
			   +str(round(objbound,2))+';'
			   +str(round(mipgap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,2))+';'
			   +str(round(tmp,2))+'\n')
            arquivo.close()

Resolvendo instancia ../data/uls/52_1.txt
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Method to value 0
Set parameter NodeMethod to value 1
Set parameter Threads to value 1
Resolvendo instancia ../data/uls/52_2.txt
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Method to value 0
Set parameter NodeMethod to value 1
Set parameter Threads to value 1
Resolvendo instancia ../data/uls/52_3.txt
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Method to value 0
Set parameter NodeMethod to value 1
Set parameter Threads to value 1
Resolvendo instancia ../data/uls/52_4.txt
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Method to value 0
Set parameter NodeMethod to value 1
Set parameter Threads to value 1
Resolvendo instancia ../data/uls/52_5.txt
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Method to va

## Tables result

In [28]:
data=f'result/uls_std_mip.csv'

df = pd.DataFrame()
df = pd.read_csv(data,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','objval','objbound','mipgap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'objval':tab["objval"].mean(),
    'objbound':tab["objbound"].mean(),
    'mipgap':tab['mipgap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"uls_mip"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["objval"] = tab["objval"].round(2)
tab["objbound"] = tab["objbound"].round(2)
tab["mipgap"] = tab["mipgap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')



In [29]:
tab

,instance,objval,objbound,mipgap,time,nodes,opt
0,../data/uls/52_1.txt,9269.20,9269.20,0.0,0.19,457.0,1
1,../data/uls/52_2.txt,9263.40,9263.40,0.0,0.23,780.0,1
2,../data/uls/52_3.txt,9179.20,9179.20,0.0,0.08,98.0,1
3,../data/uls/52_4.txt,9687.00,9687.00,0.0,0.06,1.0,1
4,../data/uls/52_5.txt,11292.00,11292.00,0.0,0.08,135.0,1
5,../data/uls/52_6.txt,11347.50,11347.50,0.0,0.05,14.0,1
6,../data/uls/52_7.txt,11157.00,11157.00,0.0,0.02,1.0,1
7,../data/uls/52_8.txt,11223.00,11223.00,0.0,0.03,1.0,1
8,../data/uls/52_9.txt,12575.80,12575.80,0.0,0.02,1.0,1
9,../data/uls/52_10.txt,12408.00,12408.00,0.0,0.02,1.0,1


In [23]:
# export to .tex format

print(
    tab[['instance','objval','objbound','mipgap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)

\begin{tabular}{lrrrrrr}
\toprule
instance & objval & objbound & mipgap & time & nodes & opt \\
\midrule
../data/uls/52_1.txt & 9269.20 & 9269.20 & 0.00 & 0.19 & 457.00 & 1 \\
../data/uls/52_2.txt & 9263.40 & 9263.40 & 0.00 & 0.23 & 780.00 & 1 \\
../data/uls/52_3.txt & 9179.20 & 9179.20 & 0.00 & 0.08 & 98.00 & 1 \\
../data/uls/52_4.txt & 9687.00 & 9687.00 & 0.00 & 0.06 & 1.00 & 1 \\
../data/uls/52_5.txt & 11292.00 & 11292.00 & 0.00 & 0.08 & 135.00 & 1 \\
../data/uls/52_6.txt & 11347.50 & 11347.50 & 0.00 & 0.05 & 14.00 & 1 \\
../data/uls/52_7.txt & 11157.00 & 11157.00 & 0.00 & 0.02 & 1.00 & 1 \\
../data/uls/52_8.txt & 11223.00 & 11223.00 & 0.00 & 0.03 & 1.00 & 1 \\
../data/uls/52_9.txt & 12575.80 & 12575.80 & 0.00 & 0.02 & 1.00 & 1 \\
../data/uls/52_10.txt & 12408.00 & 12408.00 & 0.00 & 0.02 & 1.00 & 1 \\
resume & 10740.21 & 10740.21 & 0.00 & 0.08 & 148.90 & 10 \\
\bottomrule
\end{tabular}

